# Etapa 1 - Explorando a API

## Todos os parâmetros possíveis:
            #Busca por contéudo da publicação
            'texto': texto,
            #Instituição
            'siglaTribunal': sigla_tribunal,
            #Órgão (inserir o ID do órgão. Para coletar o ID, inspecionar as páginas de https://comunica.pje.jus.br)
            'orgaoId': orgao_id,
            #Meio ('E' ou 'D')
            'meio': meio,
            #Data inicial (em string YYYY-MM-DD)
            'dataDisponibilizacaoInicio': data_inicio,
            #Data final (em string YYYY-MM-DD)
            'dataDisponibilizacaoFim': data_fim,
            #Nº de processo
            'numeroProcesso': n_processo,
            #Nome da parte
            'nomeParte': nome_parte,
            #Nome do advogado
            'nomeAdvogado': nome_adv,
            #Nº da OAB
            'numeroOab': n_oab,
            #UF da OAB
            'ufOab': uf_oab,
            'pagina': '1',
            'itensPorPagina': '5'


# Etapa 2 - Enviando Requisições (Requests) via Python

## Importação das bibliotecas necessárias

**requests** - pacote que simplifica envio de requisições HTTP <br>
**json** - utilizada para ler o conteúdo da resposta <br>
**time** - utilizada apenas para colocar uma pequena pausa entre os pedidos <br>
**math** - utilizada apenas para ajudar no cálculo do número de páginas

In [2]:
import requests
import json
import time
import math

## Função principal

In [3]:
def captura_comunica_pje(texto="", sigla_tribunal="", orgao_id="", meio="", data_inicio="", data_fim="", n_processo="", nome_parte="",
         nome_adv="", n_oab="", uf_oab=""):
    #definindo headers similares a de navegador
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
               'Accept': 'application/json, text/plain, */*',
               'Accept-Encoding': 'gzip, deflate, br, zstd'}
    params = {
            #Teor da comunicação
            'texto': texto,
            #Instituição
            'siglaTribunal': sigla_tribunal,
            #Órgão (inserir o ID do órgão. Para coletar o ID, inspecionar as páginas de https://comunica.pje.jus.br)
            'orgaoId': orgao_id,
            #Meio ('E' ou 'D')
            'meio': meio,
            #Data inicial (em string YYYY-MM-DD)
            'dataDisponibilizacaoInicio': data_inicio,
            #Data final (em string YYYY-MM-DD)
            'dataDisponibilizacaoFim': data_fim,
            #Nº de processo
            'numeroProcesso': n_processo,
            #Nome da parte
            'nomeParte': nome_parte,
            #Nome do advogado
            'nomeAdvogado': nome_adv,
            #Nº da OAB
            'numeroOab': n_oab,
            #UF da OAB
            'ufOab': uf_oab,
            'pagina': '1',
            'itensPorPagina': '5'
              }
   
    #deixando apenas os parâmetros fornecidos, para não enviar os outros como strings vazias
    params_filtrados = {k:v for k,v in params.items() if v}
    url = 'https://comunicaapi.pje.jus.br/api/v1/comunicacao'
    r = requests.get(url=url, auth=('user', 'pass'), headers=headers, params=params_filtrados)
    #for key, value in r.request.headers.items():
        #print(f"{key}: {value}")
    data = r.content.decode('utf-8')
    data_json = json.loads(data)
    #print(data_json['status'])
    if data_json['status'] != "success":
        print(f"ERRO NA PÁGINA {params['pagina']}")
    count = data_json['count']
    print(f"Número de itens = {count}")
    #são até 5 itens por página, portanto, número de páginas equivale ao 'ceiling' da divisão do n. de itens por 5.
    n_paginas = math.ceil(count / 5)
    print(f"Número de páginas = {n_paginas}")
    resultados = []
    for item in data_json['items']:
        print(item)
    for _ in range (2, n_paginas+1):
        params_filtrados['pagina'] = f"{_}"
        r = requests.get(url=url, auth=('user', 'pass'), headers=headers, params=params_filtrados)
        #print(r.url)
        data = r.content.decode('utf-8')
        data_json = json.loads(data)
        if data_json['status'] != "success":
            print(f"ERRO NA PÁGINA {params['pagina']}")
        for item in data_json['items']:
            print(item)
        time.sleep(0.5)
    return 

## Testando

In [ ]:
captura_comunica_pje(texto="", sigla_tribunal="TJMT", orgao_id="", meio="", data_inicio="2025-07-09", data_fim="2025-07-09", n_processo="", nome_parte="",
         nome_adv="SAULO NIEDERLE PEREIRA", n_oab="", uf_oab="")
